## Pipeline for upserting data to pinecone

<img src="../pipeline.png" alt="Alt text" style="background-color: white">

# Step 1
user_input = "What is meaning of life?"

source_data= retrive_source(user_input)[4]

prompt_sourcedata = "Looking at the source data, respond..."

sourcedata_response = LLM(prompt_sourcedata(user_input, source_data))

# Step 2
past_chat_context = retrieve_pastchat(sourcedata_response )[4] not in session.id

prompt_convdata = "You are now examining.. Past conversations have explored. "

conversation_response= LLM(prompt_convdata (sourcedata_response, past_chat_context))

# Step 3
final_response = sourcedata_resonse + conversation_response 


### Streamlit ###

User: Question
AI: final_response 

upsert_vectorstore(user: question, AI: final_response, session_id)

st.write(orignal_data)
st.write(past_chat_context)

In [1]:
from dotenv import load_dotenv
from datetime import datetime, timezone
import os

from data_chunking import datachunk
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint as HuggingFaceHub
from langchain_community.vectorstores.pinecone import Pinecone
from pinecone import Pinecone as pc

class chatbot:
    def __init__(self, repo_id=None, temperature=0.8, prompt_sourcedata=None, prompt_conv=None, user_name=None, session_id=None):
        self.embeddings = HuggingFaceEmbeddings()
        self.index_name = "botcon"
        self.pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)
        
        # Self-assign parameters
        self.user_name = user_name
        self.session_id = session_id
        
        self.temperature = temperature
        self.repo_id = repo_id
        
        # Instantiate the LLM
        self.llm = HuggingFaceHub(
            repo_id=self.repo_id,
            temperature=temperature,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

    # Renamed method
    def default_prompt_sourcedata(self, chat_history, original_data, user_input, user_name):
        return f"""You are an assistant associated with the artwork Carte de Continuonus by Helena Nymann. This project invites people viewing the artwork to respond to the question: "What do you want the future to remember?" The data consists of their name, location, and the response they entered to the question, which is the data we are most interested in. You are now assisting the user "{user_name}" with their query: "{user_input}". Below is the relevant data submitted to the continuonus artwork that was retrieved from the database for this query: "{original_data}". Based on this data, provide a concise answer to the user’s question in 1-3 sentences. The previous chat history for this session so far is: {chat_history} Your response:"""

    def default_prompt_conv(self, chat_history, user_input, llm_response, past_chat, user_name):
            return f"""You are an assistant observing conversations between the user and another LLM regarding statements submitted by viewers of the “Carte de Continuonus” artwork, by Helene Nyman. All interactions between people and the LLM are recorded and stored in your database. When people ask questions about the data, you get the question and the answer from the LLM. You use that data to search your database of past conversations for conversations that might be related. You will create a summary of those past conversations no longer than 4 sentences. Your summary should mention the name of the person involved in the past conversations, so that if the user wants to, they can follow up with them.
        Here is the last question asked by the user in this session: "{user_name}" asked: {user_input}
        Here is what the LLM you are watching responded with: “{llm_response}”
        Here is relevant data from past conversations that is relevant: {past_chat}
        Here is the chat history for this session, so that your response can be aware of the context: {chat_history}
        Your response: """

   # Method to retrieve documents from Pinecone index while excluding a specific session_id
    def retrieve_docs(self, input, index, excluded_session_id=None, k=5):
        # Retrieve past conversation data
        docsearch = Pinecone.from_existing_index(index, self.embeddings)
        
        if index == "bdc-interaction-data":
            # Add metadata filter to exclude the given session_id
            search_kwargs = {
                "k": k,
                "filter": {
                    "session_id": {"$ne": excluded_session_id}
                }
            }
        else:
            search_kwargs = {
                "k": k
            }
            
        retriever = docsearch.as_retriever(search_kwargs=search_kwargs)
        docs = retriever.invoke(input)
        
        return docs

    # Method to generate response from LLM
    def get_llm_response(self, prompt):
        try:
            response = self.llm.invoke(prompt)
            return response
        except Exception as e:
            print(f"Error invoking LLM: {e}")
            return "Error occurred while generating response"
    
    # Method to format the context
    def format_context(self, documents, chat=False):
        if chat == False:
            context = ""
        
            # Use enumerate to automatically add speaker numbers
            for idx, doc in enumerate(documents, start=1):
                # Extract metadata fields
                metadata = doc.metadata
                sender_name = metadata.get("sender_name", "Unknown Speaker")
                location = metadata.get("location", "Unknown Location")
                date = metadata.get("date", "Unknown Date")
                page_content = doc.page_content

                # Format each document's content with its metadata
                context += (
                    f"Person {idx}: {sender_name}\n"
                    f"Location: {location}\n"
                    f"Date: {date}\n"
                    f"Content: {page_content}\n\n"
                )
        
        else:
            context = ""
            # Use enumerate to automatically add session numbers
            for idx, doc in enumerate(documents, start=1):
                # Extract metadata fields
                metadata = doc.metadata
                user_name = metadata.get("user_name", "Unknown User")
                user_question = metadata.get("user_question", "Unknown Question")
                ai_output = metadata.get("ai_output", "Unknown Response")
                session_id = metadata.get("session_id", "Unknown Session ID")
                date = metadata.get("date", "Unknown Date")

                # Format each document's content with its metadata
                context += (
                    f'User name: {user_name}\n'
                    f'Chat session {idx}: {session_id}\n'
                    f'User Question: "{user_question}"\n'
                    f'AI Response: "{ai_output}"\n'
                    f"Date: {date}\n\n"
                )
        return context

    # Method to upsert data to Pinecone index
    def upsert_vectorstore(self, user_input, ai_output, user_name, user_location, session_id):
        # Pinecone index for chat data
        pinecone_instance_chat = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=self.embeddings)
        index_name = "bdc-interaction-data"
        environment = "gcp-starter"
        
        index = pinecone_instance_chat.Index(index_name, environment=environment)
        
        date_id = datetime.now(timezone.utc).isoformat()
        
        embedding = self.embeddings.embed_documents([user_input + ai_output])[0]
        
        # Upsert the summary embedding to the Pinecone index with metadata, including timestamp
        index.upsert(vectors=[
            {
                'id': date_id,
                'values': embedding,
                'metadata': {
                    "user_question": user_input,
                    "ai_output": ai_output,
                    "user_name": user_name,
                    "session_id": session_id,
                    "date": datetime.now(timezone.utc).isoformat(),
                    "user_location": user_location, 
                    "text": f"User input: {user_input}, AI output: {ai_output}"
                }
            }
        ])

    def pipeline(self, user_input, user_name, session_id, user_location, chat_history=None):
        # Step 0: Add chat history to the context
        if chat_history:
            chat_history = chat_history + "\n\n"
        else:
            chat_history = ""
        
        # Step 1: Retrieve source data
        source_data = self.retrieve_docs(user_input, "botcon")
        formatted_source_data = self.format_context(source_data)

        # Step 2: Generate LLM response from source data
        sourcedata_response = self.get_llm_response(self.default_prompt_sourcedata(chat_history=chat_history, original_data = formatted_source_data, user_input = user_input, user_name=user_name))

        # Step 3: Retrieve past chat context
        past_chat_context = self.retrieve_docs(sourcedata_response, "bdc-interaction-data", session_id)
        formatted_chat_context = self.format_context(past_chat_context, chat=True)
        
        # Step 5: Generate LLM response for conversation context, now considering combined chat history
        conversation_response = self.get_llm_response(self.default_prompt_conv(chat_history=chat_history, user_input=user_input, llm_response=sourcedata_response, past_chat=formatted_chat_context, user_name=user_name))

        # Step 6: Combine the responses
        ai_output = f"{sourcedata_response}\n\n{conversation_response}"
        
        print("AI Output: ", ai_output)
        print("sourcedaa_response: ", sourcedata_response)
        print("conversation_response: ", conversation_response)

        # Upsert to vector store
        self.upsert_vectorstore(user_input, ai_output, user_name, user_location, session_id)

        # Return a dictionary containing all relevant information
        return {
            "ai_output": ai_output,
            "source_data": source_data,
            "past_chat_context": past_chat_context
        }

c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
cb = chatbot(repo_id="CohereForAI/c4ai-command-r-v01")

c:\Users\szh\Desktop\EER\Bot-de-Continuonus\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [19]:
cb.pipeline("What is the meaning of life?", user_name="Alice", session_id="123", user_location="New York")

Error invoking LLM:  (Request ID: vZi43i9CkaAm7I7jpXFlW)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/CohereForAI/c4ai-command-r-v01.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model CohereForAI/c4ai-command-r-v01 is too large to be loaded automatically (69GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).
Error invoking LLM:  (Request ID: YxQms0ePgRk21WeELHuiI)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/CohereForAI/c4ai-command-r-v01.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model CohereForAI/c4ai-command-r-v01 is too large to be loaded automatically (69GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).
AI Output:  Erro

{'ai_output': 'Error occurred while generating response\n\nError occurred while generating response',
 'source_data': [Document(metadata={'emotions': ['Joy', 'Surprise'], 'location': 'Copenhagen', 'sender_name': 'Riccardo', 'seq_num': 430.0, 'slug': 'zKp1i20Uj3yA', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-06-12T18:39:43+00:00'}, page_content='To live life at its fullest'),
  Document(metadata={'emotions': ['Trust', 'Love', 'Surprise', 'Trust'], 'location': '', 'sender_name': 'Leo', 'seq_num': 576.0, 'slug': 'oiNL4z7x51L2', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-06-20T17:13:51+00:00'}, page_content='to be happy and enjoy every little thing!'),
  Document(metadata={'emotions': ['Fear'], 'location': 'Copenhagen ', 'sender_name': 'Dan', 'seq_num': 2106.0, 'slug': 'VWPXFq3uHjSz', 'source': '/work/EER-Chatbot/Bot-de-Continuonus/data/all.json', 'timestamp_ms': '2023-09-17T17:34:01+00:00'}, page_content

In [207]:
def generate_response(input_text):
    bot = cb
    result = bot.pipeline(user_input=input_text, user_name = "test", session_id="test")
    
    return result

In [208]:
result = generate_response("what is love?")

prompt for sourcedata You are an assistant with access to the artwork Carte de Continuonus by Helena Nymann, which preserves memories for future generations. You are now assisting the user "test" with their query: "what is love?". Below is the relevant data retrieved from the database for this query: "Person 1: 
Location: 
Date: Unknown Date
Content: Love for one another 

Person 2: Jofi
Location: Bremen
Date: Unknown Date
Content: Love and other feelings

Person 3: 
Location: 
Date: Unknown Date
Content: That everything is love 

Person 4: 
Location: 
Date: Unknown Date
Content: That love is the meaning

Person 5: 
Location: 
Date: Unknown Date
Content: Love is everything 

". Based on this, provide a concise response in 1-3 sentences.
prompt for conversation You are an assistant observing conversations between the user and another LLM regarding the Carte de Continuonus project.
    Most recently:
    - User "test" asked: "what is love?"
    - The LLM responded: " 

Based on the colle

In [211]:
past_chat_context = result.get("past_chat_context")

In [212]:
past_chat_context

[Document(metadata={'ai_output': ' "Sabrina, it appears that the artwork Carte de Continuonus contains memories from various individuals, mostly from Copenhagen, with brief yet profound statements. Some of the messages seem to share a common theme of appreciating life\'s brevity, while others focus on more specific activities or sentiments. These memories are left without specific dates, leaving room for interpretation."\n\n "Sabrina, this focus on appreciating life\'s brevity echoes our earlier discussions about the meaning of life and creating meaning in the future. It\'s fascinating to see how these individual memories collectively convey a sense of hope and caution for what\'s to come. "', 'date': '2024-09-24T13:34:41.730587+00:00', 'session_id': '863e99eb-493b-4392-b4ec-e48fd2183ea6', 'user_question': 'tell me more'}, page_content='User input: tell me more, AI output:  "Sabrina, it appears that the artwork Carte de Continuonus contains memories from various individuals, mostly fro

In [216]:
past_chat_context[0].metadata.get("")

In [ ]:
for idx, doc in enumerate(past_chat_context, 1):
                            with st.expander(f"User question: _\"{doc.metadata.get('user_question')}\"_", expanded=False):
                                st.markdown(f"**AI Response:** _\"{doc.metadata.get('ai_output')}\"_")
                                st.markdown(f"**Date:** {doc.metadata.get('date', 'Unknown date')}")
                                st.markdown(f"**User name:** {doc.metadata.get('user_name', 'Unknown user name')}")
                                st.markdown(f"**Location:** {doc.metadata.get('location', 'Unknown location')}")

'You are an assistant observing conversations between the user and another LLM regarding statements submitted by viewers of the “Carte de Continuonus” artwork, by Helene Nyman. All interactions between people and the LLM are recorded and stored in your database. When people ask questions about the data, you get the question and the answer from the LLM. You use that data to search your database of past conversations for conversations that might be related. You will create a summary of those past conversations no longer than 4 sentences. Your summary should mention the name of the person involved in the past conversations, so that if the user wants to, they can follow up with them.\n        Here is the last question asked by the user in this session: "user_name" asked: user_input\n        Here is what the LLM you are watching responded with: “llm_response”\n        Here is relevant data from past conversations that is relevant: past_chat\n        Here is the chat history for this session